In [19]:
import os
import re
import random
import copy
from collections import Counter
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gc
    
from gensim.models import KeyedVectors

import numpy as np
import pandas
from pandas import Series, DataFrame

import keras
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

import matplotlib.pyplot as plt
import seaborn as sns       

from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

#进行配置，使用30%的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )

In [2]:
# 文件路径
ORIGINAL_FILE_PATH = "../data/original_data/quora_duplicate_questions.tsv"

DATA_ALL_PATH = "../data/all_data.txt"

DATA_TRAIN_PATH = "../data/train.txt"
DATA_VALID_PATH = "../data/valid.txt"
DATA_TEST_PATH = "../data/test.txt"

MAPS_FILE_PATH = "../data/maps.pkl"

# Glove文件
WORD2VEC_PATH = "../data/GoogleNews-vectors-negative300.bin"

In [17]:
def load_dict(dict_path):
    with open(dict_path, 'rb') as fr:
        return pickle.load(fr)
        
def create_maps(dico):
    item2id = dico
    id2item = dict([val, key] for key, val in dico.items())
    return item2id, id2item

def read_file(fpath):
    """
    读取文件，返回
    """
    question1, question2, labels = [], [], []
    with open(fpath, 'r', encoding="utf-8") as fr:
        for line in fr.readlines():
            try:
                q, k, l = line.strip().split('\t')
            except ValueError:
                print(line)
            question1.append(q)
            question2.append(k)
            labels.append(int(l))
            
    return question1, question2, labels

def load_data(fpath, word_to_id, max_length=20):
    """
    载入数据，并将其转化为id表示
    """
    question1, question2, labels = read_file(fpath)
    
    q1_id, q2_id = [], []
    for i in range(len(question1)):
        q1= question1[i].split()
        q2 = question2[i].split()
        q1_id.append([word_to_id[x] for x in q1 if x in word_to_id])
        q2_id.append([word_to_id[x] for x in q2 if x in word_to_id])

    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x1_pad = keras.preprocessing.sequence.pad_sequences(q1_id, max_length, padding="post")
    x2_pad = keras.preprocessing.sequence.pad_sequences(q2_id, max_length, padding="post")
    labels = keras.utils.np_utils.to_categorical(labels)
    
    return x1_pad, x2_pad, labels
    
        
def load_word2vec(embedding_path, word_index, embed_dim=300): 
    """
    载入预训练好的word2vec(Google-News)
    """
    # 载入word2vec词向量
    word2vec_dict = KeyedVectors.load_word2vec_format(embedding_path,binary=True)
    
    embedding_index = dict()
    for word in word2vec_dict.wv.vocab:
        embedding_index[word] = word2vec_dict.word_vec(word)
    print('Load %s word vectors.' % len(embedding_index))
    
    all_embs = np.stack(list(embedding_index.values()))
    # emb_mean, emb_std = all_embs.mean(), all_embs.std()
    emb_mean = np.mean(all_embs, axis=0)
    vocab_size = len(word_index)
    # 初始化权重
    embedding_matrix = np.zeros((vocab_size+1, embed_dim))
    gc.collect()
    # 对权重矩阵进行赋值，未找到时用词向量平均值填充
    for word, i in word_index.items():
        if i < vocab_size + 1:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                embedding_matrix[i] = emb_mean
    del embedding_index
    return embedding_matrix

In [18]:
dico = load_dict(MAPS_FILE_PATH)
word_index, index_word = create_maps(dico)

# 准备数据
x1_train, x2_train, y_train = load_data(DATA_TRAIN_PATH, word_index)
x1_valid, x2_valid, y_valid = load_data(DATA_VALID_PATH, word_index)
x1_test, x2_test, y_test = load_data(DATA_TEST_PATH, word_index)

embedding_matrix = load_word2vec(WORD2VEC_PATH, word_index)

Load 3000000 word vectors.


In [30]:
vocab_size = len(dico)

class Settings(object):
    text1_maxlen = 20
    text2_maxlen = 20
    embedding_dim = 300
    vocab_size = vocab_size
    kernel_count = 128
    kernel_size = 3
    q_pool_size = 4
    d_pool_size = 4
    dropout_rate = 0.2
    
settings = Settings()

In [31]:
from keras.layers import Input, Embedding, Conv1D, MaxPool1D, Concatenate, Flatten, Dropout, Dense, MaxPooling1D
from keras.models import Model

# 定义网络
query = Input(name='query', shape=(settings.text1_maxlen,))
doc = Input(name='doc', shape=(settings.text2_maxlen,))


embedding = Embedding(settings.vocab_size + 1,
                      settings.embedding_dim,
                      weights=[embedding_matrix],
                      input_length=settings.text1_maxlen,
                      trainable=False)

q_embed = embedding(query)
d_embed = embedding(doc)

q_conv1 = Conv1D(settings.kernel_count, settings.kernel_size, padding='same') (q_embed)
d_conv1 = Conv1D(settings.kernel_count, settings.kernel_size, padding='same') (d_embed)

q_pool1 = MaxPooling1D(pool_size=settings.q_pool_size)(q_conv1)
d_pool1 = MaxPooling1D(pool_size=settings.d_pool_size)(d_conv1)

pool1 = Concatenate(axis=1)([q_pool1, d_pool1])

pool1_flat = Flatten()(pool1)

pool1_flat_drop = Dropout(rate=settings.dropout_rate)(pool1_flat)

out_ = Dense(2, activation='softmax')(pool1_flat_drop)

model = Model(inputs=[query, doc], outputs=out_)
model.summary()        

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 20)           0                                            
__________________________________________________________________________________________________
doc (InputLayer)                (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 20, 300)      25293600    query[0][0]                      
                                                                 doc[0][0]                        
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 20, 128)      115328      embedding_3[0][0]                
__________

In [32]:
import datetime,time
from keras.optimizers import Adam\

callback_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="../model/arci/arci-01.h5",
        monitor='val_loss',
        save_best_only=True
    )
]

adadelta=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['binary_accuracy'])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

print('Train...')

model.fit([x1_train, x2_train], y_train, epochs=100, callbacks=callback_list, validation_data=([x1_valid, x2_valid], y_valid))

Train...
Train on 323426 samples, validate on 40428 samples
Epoch 1/100
323426/323426 [==============================] - 74s 229us/step - loss: 0.5462 - acc: 0.7251 - val_loss: 0.5311 - val_acc: 0.7392
Epoch 2/100
323426/323426 [==============================] - 71s 218us/step - loss: 0.5056 - acc: 0.7535 - val_loss: 0.5078 - val_acc: 0.7496
Epoch 3/100
323426/323426 [==============================] - 70s 217us/step - loss: 0.4761 - acc: 0.7716 - val_loss: 0.5240 - val_acc: 0.7380
Epoch 4/100
323426/323426 [==============================] - 71s 218us/step - loss: 0.4531 - acc: 0.7854 - val_loss: 0.5058 - val_acc: 0.7609
Epoch 5/100
323426/323426 [==============================] - 70s 218us/step - loss: 0.4329 - acc: 0.7970 - val_loss: 0.5118 - val_acc: 0.7586
Epoch 6/100
323426/323426 [==============================] - 71s 219us/step - loss: 0.4167 - acc: 0.8057 - val_loss: 0.5226 - val_acc: 0.7493
Epoch 7/100
323426/323426 [==============================] - 70s 216us/step - loss: 0.40

Epoch 58/100
323426/323426 [==============================] - 70s 217us/step - loss: 0.2505 - acc: 0.8947 - val_loss: 0.7519 - val_acc: 0.7741
Epoch 59/100
323426/323426 [==============================] - 70s 216us/step - loss: 0.2512 - acc: 0.8946 - val_loss: 0.7509 - val_acc: 0.7584
Epoch 60/100
323426/323426 [==============================] - 69s 214us/step - loss: 0.2489 - acc: 0.8958 - val_loss: 0.7607 - val_acc: 0.7756
Epoch 61/100
323426/323426 [==============================] - 70s 215us/step - loss: 0.2481 - acc: 0.8954 - val_loss: 0.7485 - val_acc: 0.7739
Epoch 62/100
323426/323426 [==============================] - 70s 216us/step - loss: 0.2490 - acc: 0.8960 - val_loss: 0.7457 - val_acc: 0.7716
Epoch 63/100
323426/323426 [==============================] - 70s 216us/step - loss: 0.2481 - acc: 0.8959 - val_loss: 0.7616 - val_acc: 0.7722
Epoch 64/100
323426/323426 [==============================] - 69s 214us/step - loss: 0.2467 - acc: 0.8967 - val_loss: 0.7777 - val_acc: 0.7555

In [33]:
y_vec_pred = model.predict([x1_test, x2_test])

In [54]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

y_pred_shape = y_vec_pred.shape
y_pred = np.zeros(y_pred_shape)

i = 0
for item in y_vec_pred:
    if item[0] < item[1]:
        y_pred[i] = np.array([0, 1])
    else:
        y_pred[i] = np.array([1, 0])
    i += 1
    
report = precision_recall_fscore_support(y_test, y_pred, average='macro')

In [55]:
print(report)

(0.7587268891912451, 0.7448385780363961, 0.7501467419196652, None)


In [37]:
model.evaluate([x1_test, x2_test], y_test)

40429/40429 [==============================] - 3s 85us/step


[0.8268117943997522, 0.7725642484384776]